# Anime Analysis by Ben Osborn and OsbornAI

## This project consists of the scraping and creation of a dataset containing information about all anime's listed on MyAnimeList. This data is analysed, and a model is created to predict the anime's rating based on the pages features

### Imports

In [43]:
import requests
from bs4 import BeautifulSoup
import time

### Scraping and dataset creation

#### Removes the label that is included with the scraped data

In [5]:
# Change name to clean label
def parseLabel(element):
    string = element.text
    
    split_colens = string.split(':')
    removed_label = split_colens[1:]
    
    for i, label in enumerate(removed_label):
        removed_label[i] = label.replace('\n', '').strip()
    
    joined = " ".join(removed_label)
    
    return joined

In [6]:
def parseList(element): # Have to check that a tags exist for ever single page and developer
    ret_list = [a.text for a in element.find_all('a')]
    
    return ret_list

#### Scrapes the anime page and returns a row of data

In [80]:
def createRow(url):
    req = requests.get(url)
    soup = BeautifulSoup(req.content, 'html.parser')

    description = soup.find('p', itemprop='description').text # --------- Check this one

    side_panel = soup.find('td', class_='borderClass')
    side_panel_subdiv = side_panel.find('div')
    side_panel_divs = side_panel_subdiv.find_all('div')
    
    name = parseLabel(side_panel_divs[7]) # Good
    show_type = parseLabel(side_panel_divs[10]) # Good
    episodes = parseLabel(side_panel_divs[11]) # Good
    status = parseLabel(side_panel_divs[12]) # Good
    
    aired_raw = parseLabel(side_panel_divs[13]) 
    aired = [time.strftime('%d-%m-%Y', time.strptime(date.strip().replace(',', ''), '%b %d %Y')) for date in aired_raw.split("to")]
    aired_start = aired[0] # Good
    aired_end = aired[1] # Good
    
    broadcast_raw = parseLabel(side_panel_divs[15])
    broadcast_split = [element.strip() for element in broadcast_raw.split('at')]
    broadcast_split[0] = broadcast_split[0][:-1]
    broadcast_split[1] = ":".join(broadcast_split[1][:5].split(' '))
    broadcast = " ".join(broadcast_split) # Good
    
    producers = parseList(side_panel_divs[16]) # Good
    licensors = parseList(side_panel_divs[17]) # Good
    studios = parseList(side_panel_divs[18]) # Good
    source = parseLabel(side_panel_divs[19]) # Good
    genres = parseList(side_panel_divs[20]) # Good
    
    duration_raw = parseLabel(side_panel_divs[21])
    duration = duration_raw.split(' ')[0] # Good
    
    rating_raw = parseLabel(side_panel_divs[22])
    rating = rating_raw.split(' ')[0] # Good
    
    score = [part.text for part in side_panel_divs[23].find_all('span')][1:] # Good
        
    members = "".join(parseLabel(side_panel_divs[28]).split(',')) # Good
    favourites = "".join(parseLabel(side_panel_divs[29]).split(',')) # Good
    
createRow("https://myanimelist.net/anime/5114/Fullmetal_Alchemist__Brotherhood")

167574


#### Go through the amount of pages specified then scrape the information for each show, then store them to a CSV file

In [27]:
num_pages = 1 # Roughly 300 max, test on 0 for now (Which is the first page)

# This should end up being multithreaded to do more requests at once

for i in range(num_pages):
#   This section here forcuses on grabbing the link for each page
    url_page = f"https://myanimelist.net/topanime.php?limit={i*50}"
    req_list = requests.get(url_page)
    soup_list = BeautifulSoup(req_list.content, 'html.parser')
    shows = soup.find_all('tr', class_='ranking-list')
    
#   This section is going to go to each show in the list and copy the data into it's own section
    for i in range(50):
        link = shows[i].find('a').get('href')
        req_show = requests.get(link)
        soup_show = BeautifulSoup(req_show.content, 'html.parser')

#         I want to store the name, and all of the information and statistics down the side as a csv
#         Watch out because some of the data values will contain commas

NameError: name 'soup' is not defined